In [1]:
import pandas as pd
import os 
import numpy as np
from TSMixerAPI import TSMixer
from metrics import metric

### Sim Dataset

In [32]:
lengths = [90,420]
num_series = [50,300]
strctures = ['stationary','trend']
int_type = 'hom'
data_type = 'sim'
seasonality_period = 30
pred_len = 24
data_root_path = base_dir = os.path.join(os.getcwd(),'../../../')

In [ ]:
model = TSMixer()
model.compile(learning_rate=1e-4, loss='mae')
for iter in range(model.args.itr):
    for len in lengths:
        for series in num_series:
            for strcture in strctures:
                data_name = data_type + '_' + str(len) + '_' + str(series) + '_' + strcture + '_' + int_type
                print(f"Training {TSMixer.__name__} on {data_name} with pred_len {pred_len}")

                model.fit(
                iter = iter + 1,
                data_type = data_type,
                data_name=data_name,
                data_root_path=data_root_path,
                batch_size=32,
                epochs=100,
                seasonality_period = seasonality_period,
                pred_len=pred_len,
                features = 'M',
                state_index = None
                )

                predictions = model.predict()

                print(f'\nMoving to next...\n')

### Electricity Price Dataset

In [9]:
seasonality_period = 12
seq_len = 12
pred_len = 24
data_type = 'elec_price'
data_root_path = os.path.join(os.getcwd(),'../../../')
data_name = 'priceMT' 
features = 'MS'
setting = f"TSMixer_{data_type}_{data_name}_{features}_sl{seq_len}_pl{pred_len}"

In [ ]:
model = TSMixer()
model.compile(learning_rate=1e-4, loss='mae')
for i in range (1):
    for iter in range(model.args.itr):
        
    
        print(f"Training {TSMixer.__name__} on {data_name} with pred_len {pred_len}")

        model.fit(
        iter = iter + 1,
        data_type = data_type,
        data_name=data_name,
        data_root_path=data_root_path,
        batch_size=32,
        epochs=100,
        seasonality_period = seasonality_period,
        pred_len=pred_len,
        features = 'MS',
        target="TARGET",
        state_index = i
        )

        predictions_iter = model.predict()
        predictions = pd.concat(predictions_iter, axis=1)

    print(f'\nMoving to next...\n')
        

In [4]:
control = ['AK', 'AL', 'AR', 'AZ', 'CO', 'DE', 'ID', 'FL', 'GA', 'HI', 'IA', 'IN', 'KS', 'KY', 'LA', 
            'ME', 'MN', 'MI', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NM', 'NV', 'OH', 'OK', 'OR',
            'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [23]:
df_raw = pd.read_csv(data_root_path+ "data/" + data_type + "/" + data_name + "_" + \
                                  "full_table" + ".csv")
trues = df_raw.iloc[len(df_raw)- pred_len:,:]
trues_control = np.array(df_raw.loc[len(df_raw)- pred_len:,control])        
df_a_control =  df_raw.loc[:(len(df_raw)- pred_len)  ,control] 

In [6]:
preds = pd.read_csv("/Users/orshahar/Desktop/School/causality_analysis/results/elec_price/tsmixer/forecasts/TSMixer_elec_price_priceMT_MS_sl12_pl24_preds.csv").iloc[:,1:51]
preds_control = np.array(preds.loc[:,control])
preds_for_errors_control = np.array(preds_control)


In [24]:
metric_folder_path = data_root_path + "/results/" + data_type + "/" + "tsmixer/"  +  "metrics" + "/"
mae_control, mse_control, rmse_control, smape_control, mase_control = metric( 
        preds_for_errors_control, trues_control ,df_a_control, seasonality_period)

all_metrics_control = [mae_control, mse_control, rmse_control, smape_control, mase_control]
metric_list = ['mae', 'mse', 'rmse', 'smape', 'mase']
metric_df_control = pd.DataFrame([all_metrics_control], columns=metric_list)
metric_df_control.to_csv(metric_folder_path + setting + "_" + "metrics_control.csv", index = False)

In [25]:
metric_df_control

,mae,mse,rmse,smape,mase
0,0.403657,0.357113,0.597589,0.051772,2.191313
